# Combining and grouping DataFrames

In [1]:
import numpy as np
import pandas as pd

Two of the most important operations one can do with DataFrames is 1) combine multiple sources of information and 2) efficiently group and summarize information. Both of these topics are vast and here again, we ony present a few important approaches.

## Concatenation

The simplest case where we need to combine multiple sources of information is if those sources are of the same "type", i.e. they have the same columns but different entries. This is exactly the situation we have from our analysis: lots of CSV files with similar contents. In that case we can just "glue" or concatenate them. Let's load two of them:

In [2]:
nuclei1 = pd.read_csv('../exports/19838_1252_F8_1_in.csv')
nuclei1.head(4)

,label,area,mean_intensity
0,1,5629,28.214070
1,2,9904,44.429826
2,4,15070,53.126078
3,5,20884,49.792856


In [3]:
nuclei2 = pd.read_csv('../exports/24138_196_F7_2_in.csv')
nuclei2.head(4)

,label,area,mean_intensity
0,1,35241,59.340342
1,2,29807,59.633610
2,3,32710,55.387435
3,4,33727,76.543956


As both tables have the same columns, we can concatenate them:

In [4]:
all_nuclei = pd.concat([nuclei1,nuclei2])

In [5]:
all_nuclei

,label,area,mean_intensity
0,1,5629,28.214070
1,2,9904,44.429826
2,4,15070,53.126078
3,5,20884,49.792856
4,6,12972,42.911116
5,7,16068,54.610904
6,8,27912,52.343007
7,9,26131,60.766178
8,10,28071,58.830430
9,11,16176,54.782517


## Joining two tables

A slightly more complex case is if we have two datasets with (almost) the same items, in our case nuclei ids, but with different information that we want to combine. This is exactly our situations with the two tables corresponding to intensities inside the nuclei and on their borders. We want for each nucleus (characterized by a label) to have the mean intensity both inside and on the border.

Here we import the pair of tables corresponding to one image and rename the intensity columns for clarity:

In [6]:
nuclei1 = pd.read_csv('../exports/19838_1252_F8_1_in.csv')
nuclei1.rename(columns={'mean_intensity': 'intensity_in'}, inplace=True)
nuclei1.head(4)

,label,area,intensity_in
0,1,5629,28.214070
1,2,9904,44.429826
2,4,15070,53.126078
3,5,20884,49.792856


In [7]:
nuclei2 = pd.read_csv('../exports/19838_1252_F8_1_out.csv')
nuclei2.rename(columns={'mean_intensity': 'intensity_out'}, inplace=True)
nuclei2.head(4)

,label,area,intensity_out
0,1,3675,19.604354
1,2,4484,27.112400
2,3,52,31.884615
3,4,5746,31.962931


Note that not all labels appear in both tables. As we used erosion to create a band around nuclei, small nuclei just disappeared in the process. Hence in the "in" data we are for example missing the label ```3```. We see in the "out" table that this corresponded to a tiny nucleus.

If we consider the statement: **merge table A and table B** with table A being on the left and B on the right, we have the four possibilities illustrated below:
- **left**: keep elements that appear in both A and B or only in A but not those only appearing in B
- **right**: keep elements that appear in both A and B or only in B but not those only appearing in A
- **inner**: keep elements that appear in both A and B
- **outer**: keep all elements

In [8]:
from IPython.display import Image
Image(url='https://github.com/guiwitz/ISDAwPython_day2/raw/master/images/left_right.jpeg',width=700)

## ```merge``` function

To achieve this joining of two tables, we use the ```pd.merge```function. The merge function needs the following information:
- two DataFrames to merge: ```nuclei1``` and ``` nuclei2```
- an indication of which columns to take as reference, i.e. which columns indicate which nucleus is which. In our case it is ```label``` but it could be a differently named column in each table
- what type of merging is desired (left, inner etc.)

Let's test this in ```inner``` mode:

In [9]:
pd.merge(
    left=nuclei1,
    right=nuclei2,
    left_on='label',
    right_on='label',
    how='inner')

,label,area_x,intensity_in,area_y,intensity_out
0,1,5629,28.214070,3675,19.604354
1,2,9904,44.429826,4484,27.112400
2,4,15070,53.126078,5746,31.962931
3,5,20884,49.792856,7425,31.777778
4,6,12972,42.911116,3176,28.912154
5,7,16068,54.610904,5261,38.035925
6,8,27912,52.343007,6906,37.743412
7,9,26131,60.766178,6183,44.668769
8,10,28071,58.830430,6505,41.293928
9,11,16176,54.782517,6131,26.763986


We see above that all the information has been merged. In cases where we have twice the same column name, e.g. ```area```, Pandas automatically renames the columns, here in ```area_x``` and ```area_y```.

We could also have said that we want to keep all data, including those with incomplete information:

In [10]:
pd.merge(
    left=nuclei1,
    right=nuclei2,
    left_on='label',
    right_on='label',
    how='outer')

,label,area_x,intensity_in,area_y,intensity_out
0,1,5629.0,28.214070,3675,19.604354
1,2,9904.0,44.429826,4484,27.112400
2,4,15070.0,53.126078,5746,31.962931
3,5,20884.0,49.792856,7425,31.777778
4,6,12972.0,42.911116,3176,28.912154
5,7,16068.0,54.610904,5261,38.035925
6,8,27912.0,52.343007,6906,37.743412
7,9,26131.0,60.766178,6183,44.668769
8,10,28071.0,58.830430,6505,41.293928
9,11,16176.0,54.782517,6131,26.763986


We see that we recovered the label ```3``` but of course the information about nuclei inside is missing and replaced with ```NaN``` values.

## Grouping

Very often when you want to calculate some statistics on a dataset you need to group some of the data. For example we can imagine that we just concatenated the tables corresponding to nuclei inside and border, just taking care of adding a column specifying the type of info:

In [11]:
nuclei1 = pd.read_csv('../exports/19838_1252_F8_1_in.csv')
nuclei2 = pd.read_csv('../exports/19838_1252_F8_1_out.csv')

nuclei1['image_type'] = 'in'
nuclei2['image_type'] = 'out'

all_nuclei = pd.concat([nuclei1, nuclei2])

In [12]:
nuclei1.head(4)

,label,area,mean_intensity,image_type
0,1,5629,28.214070,in
1,2,9904,44.429826,in
2,4,15070,53.126078,in
3,5,20884,49.792856,in


In [13]:
nuclei2.head(4)

,label,area,mean_intensity,image_type
0,1,3675,19.604354,out
1,2,4484,27.112400,out
2,3,52,31.884615,out
3,4,5746,31.962931,out


In [14]:
all_nuclei

,label,area,mean_intensity,image_type
0,1,5629,28.214070,in
1,2,9904,44.429826,in
2,4,15070,53.126078,in
3,5,20884,49.792856,in
4,6,12972,42.911116,in
5,7,16068,54.610904,in
6,8,27912,52.343007,in
7,9,26131,60.766178,in
8,10,28071,58.830430,in
9,11,16176,54.782517,in


Now that we have this table (probably the result of a much larger concatenation), we would like to compute the overall mean intensity, but still split the data **by type**. We could of course manually select columns and compute averages e.g. :

In [15]:
all_nuclei[all_nuclei.image_type == 'in']['mean_intensity'].mean()

50.977129452652825

However depending on data complexity this can quickly become unwidely.

Instead we can use the ```groupy``` method. To use it we just have to specify which column we want to use for grouping and Pandas will find in there all possible values and then group all elements according to those values:

In [16]:
grouped_nuclei = all_nuclei.groupby('image_type')

In [17]:
grouped_nuclei

The output is not directly visible. It's a collection of tables grouped according to species. Many functions allow us to get one of these subgroups, e.g.:

In [18]:
grouped_nuclei.get_group('in')

,label,area,mean_intensity,image_type
0,1,5629,28.214070,in
1,2,9904,44.429826,in
2,4,15070,53.126078,in
3,5,20884,49.792856,in
4,6,12972,42.911116,in
5,7,16068,54.610904,in
6,8,27912,52.343007,in
7,9,26131,60.766178,in
8,10,28071,58.830430,in
9,11,16176,54.782517,in


However we won't go here through the details of the group data structure. What we can simply do is magically apply functions directly on the grouped object. For example to get the mean:

In [19]:
grouped_nuclei.mean()

,label,area,mean_intensity
image_type,,,
in,6.818182,17970.0,50.977129
out,6.500000,5069.5,33.172806


As you can see Pandas automatically computes the mean for each category and each column. The output then is a DataFrame where each line corresponds to a given category. One can push this further by using e.g. multiple columns for grouping, but this goes beyond the present course.

## Exercises

1. Load a first DataFrame that contains information about covid19 vaccination locations: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/locations.csv 
2. Load a second DataFrame which contains daily information about vaccination for every country: https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv
3. Create a sub-dataframe of the table in (1) that only contains the fields ```location``` and ```vaccines```.
4. Merge the two tables using the ```location``` field as key for merging. Use once ```left``` merge and once ```right``` marge.
5. Do you see a difference in the result?
6. Using the table from (1), group the data by ```location``` and calculate the average number of ```daily_vaccinations```